### 世紀風電 data cleaning

Step 1.各assembly要加註到各工站裡(ex.TP + Upper JKT -> TP + Upper JKT_Full JKT Assembly)  
Step 2.複製檔案路徑去讀取檔案  
Step 3.將data_d調整為資料提供的日期

In [27]:
import pandas as pd
from datetime import date

# 讀取檔案
df = pd.read_excel(r"D:\Users\c1635\Desktop\世紀0503.xlsx")

# Start cleaning data 資料取得日期設定
# data_d = pd.DatetimeIndex([date(2023, 4, 12).isoformat()]*len(df)) #裡面數字需要調整
# df.insert(loc=0, column="Date", value=data_d)
# 先把不要的欄位刪除
# df = df.drop(columns="Activity Name")

# 不同階段分開成 DataFrame, lot->階段, assembly->組裝階段, site->工作站
lot = df[df["Activity ID"].str.match("  [A-Z]") == True]
assembly = df[df["Activity ID"].str.match("    [A-Z]") == True]
site = df[df["Activity ID"].str.match("      [A-Z]") == True]

In [28]:
# 將日期文字格式改為日期格式
trans = ["BL Project Start", "BL Project Finish", "Actual Start", "Actual Finish"]
for i in trans:
    df[i] = pd.to_datetime(df[i])

In [29]:
df.loc[393:395]

,Activity ID,Activity Name,BL Project Duration,BL Project Start,BL Project Finish,Actual Duration,Actual Start,Actual Finish
393,CF34,NaN,0.0,2023-06-12,2023-06-12,0,NaT,NaT


In [30]:
# 將工站作業階段取出到 list 內, 找出 index 後放到 Jktid row(index+1) 
states = []
for state in site["Activity ID"]:
    states.append(state)

# 找出工站的 index，再放到 index+1 row 
for i in range(0,len(states)):
    num = df.index[df["Activity ID"] == states[i]]
    df.loc[num+1, "temp"] = states[i]
    new = df["temp"].str.split("_", n = 1, expand = True)

    df["Work State"] = new[0]
    df["Assembly State"] = new[1]


KeyError: '[394] not in index'

In [31]:
# 將階段取出到 list 內, 找出 index 後放到 Jktid row(index+3) 
phases = []
for phase in lot["Activity ID"]:
    phases.append(phase)

for i in range(0, len(phases)):
    num = df.index[df["Activity ID"] == phases[i]]
    
    df.loc[num+3, "Phase"] = phases[i]

In [32]:
df2 = df.drop(["temp"], axis=1)
jkt = df2[df2["Activity ID"].str.match("        \w") == True]
stripls = ["Activity ID", "Work State","Assembly State", "Phase"]
for i in stripls:
    jkt[i] = jkt[i].str.lstrip()
# jkt["Work State"] = jkt["Work State"].str.lstrip()
# jkt["Assembly State"] = jkt["Assembly State"].str.lstrip()
# jkt["Phase"] = jkt["Phase"].str.lstrip()

D:\Users\c1635\AppData\Local\Temp\ipykernel_9656\1061450890.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jkt[i] = jkt[i].str.lstrip()


In [33]:
# jkt = jkt.drop(columns=["Original Duration", "Start","Finish"])

In [34]:
import xlwings as xw
wb = xw.Book()

stages = ["lot", "assembly", "site", "jkt"]
for stage in stages:
    wb.sheets.add(stage)

wb.sheets["lot"].range("A1").options(index=False).value = lot
wb.sheets["assembly"].range("A1").options(index=False).value = assembly
wb.sheets["site"].range("A1").options(index=False).value = site
wb.sheets["jkt"].range("A1").options(index=False).value = jkt

### 興達海基 data cleaning

原始資料 -> 一個工站一張sheet -> 合併成一張sheet -> 總表update興達new資訊  

Step 1.複製excel路徑  
Step 2.將合併好的Workbook另存為【興達new】在桌面上
Step 3.篩選出-符號，delete符號，這樣才不會噴Error
Step 3.將BI data內的興達海基，Ctrl+T選取後，改資料表名稱為【raw_data】  
Step 4.興達海基.xlsx[大表] 進行重新整理  
Step 5.記得 Save


In [1]:
import pandas as pd
import xlwings as xw

wb = xw.Book(r"D:\2023_OffshoreWindPlan\# data\# 中心廠生產進度資料\興達海基\230413\SDMS__LV4_20230413.xlsx") #貼上連結
data = []
for num in range(0,len(wb.sheets)):
    sheet = wb.sheets[num]
    last_row = sheet.range("A7").end("down").row
    for i in range(7, last_row+1):
        data_row = sheet.range(f"A{i}:G{i}").value
        data.append(data_row)
        
df = pd.DataFrame(data)
df.columns = ["Activity Name",
              "Plan Start",
              "Plan Finish",
              "Actual Start",
              "Actual Finish",
              "Plan Duration",
              "Actual Duration"
             ]

new_wb = xw.Book()
new_sheet = new_wb.sheets[0]
new_sheet.range("A1").options(index=False).value = df

### Update data values

In [4]:
import pandas as pd
import xlwings as xw

original_df = pd.read_excel(r"D:\2023_OffshoreWindPlan\# data\$ BIdata\興達海基.xlsx")
new_df = pd.read_excel(r"D:\Users\c1635\Desktop\興達new.xlsx")

trans = ["Plan Start", "Plan Finish", "Actual Start", "Actual Finish"]
for i in trans:
    original_df[i] = pd.to_datetime(original_df[i])
    new_df[i] = pd.to_datetime(new_df[i])
    
original_df.set_index("Original Acitity Name",inplace=True)
original_df.update(new_df.set_index('Activity Name'))
original_df.reset_index()



wb = xw.Book(r"D:\2023_OffshoreWindPlan\# data\$ BIdata\興達海基.xlsx")
sheet = wb.sheets[0]

sheet.range("A1").value = original_df